In [1]:
import pandas as pd
import numpy as np
from os import listdir
import openpyxl
from datetime import datetime
pd.options.mode.chained_assignment = None
import geopandas as gpd

In [2]:
# Set location 
location = "home"

if location == "home":
    location_data = "C:/Users/Lara/Work/SFN/Forms"
elif location == "work":
    location_data = "O:/SNF/SNF_mapping_2025/Excel_forms"

## Prep Postcode district data

In [27]:
gdf = gpd.read_file("G:/Postcode_Areas/Districts.shp")
gdf['geometry'] = gdf['geometry'].representative_point()
gdf['Long'] = gdf.geometry.x
gdf['Lat'] = gdf.geometry.y
df_postcode_lookup = gdf.drop("geometry", axis = 1)
df_postcode_lookup = df_postcode_lookup.rename(columns= {"name":"Postcode"})

## Prep forms data and add in postcode district

In [32]:
# Set up dataset
df_total = pd.DataFrame({ "Farm number":[],"Network name":[], "Start year":[], "Web link":[], "Contact email":[],"Postcode":[], "Category":[], "Activity":[] })
    
# Loop through files in forms folder
for file in listdir(location_data):
    # Only read in excel files
    if file.endswith(".xlsx"):
        print(file)

        # Read in data
        df = pd.read_excel(f"{location_data}/{file}")
        
        # Check data layout seems to be correct
        assert  df.iloc[53,0] == "Start year of farm's network membership", "Data is not in correct format"
        assert  df.iloc[12,2] == "Farm", "Data is not in correct format"
        assert  df.iloc[15,0] == "Postcode district eg CH8, SN10, M9", "Data is not in correct format"
    
        # Check if form contains network web link
        if pd.isna(df.iloc[4,2])==False:
            # If form contains weblink, need to use openxyl to access
            wb = openpyxl.load_workbook(f"{location_data}/{file}")
            sheets = wb.sheetnames
            ws = wb[sheets[0]]
            network_web_link = ws.cell(row=6, column=3).hyperlink.target
        else:
            network_web_link = ""

        # Set network name
        if pd.isna(df.iloc[4,1])==False:
            network_name = df.iloc[4,1]
        else:
            network_name = ""
        # Set network contact email
        if pd.isna(df.iloc[4,3])==False:
            network_email = df.iloc[4,3]
        else:
            network_email = ""
        

        # Get list of start years for each farm
        df_year = df[df.iloc[:,0] == "Start year of farm's network membership"]
        year_list = df_year.iloc[:,2:].values.flatten().tolist()
        # Check no missing start year (removed for now as one form does not have start years)
        # assert pd.isna(year_list).any() == False, "Missing start year"

        # Get list of farm postcodes for each farm
        df_postcode = df[df.iloc[:,0] == "Postcode district eg CH8, SN10, M9"]
        postcode_list = df_postcode.iloc[:,2:].values.flatten().tolist()
        # Check no missing postcode
        assert pd.isna(postcode_list).any() == False, "Missing postcode"

        # Select rows with information we need
        df_clean = df.iloc[13:52,:]
        df_clean.iloc[0,0] = "Category"
        # Set first row as column headers and remove that row
        df_clean.columns = df_clean.iloc[0]
        df_clean = df_clean.iloc[1:]
        df_clean.iloc[:,0]  = df_clean.iloc[:,0].ffill(axis = 0)
        # Remove 2 uneeded rows in middle of dataframe
        df_clean= df_clean.iloc[4:,:]

        # Get list of farm numbers
        farm_number_list = df_clean.iloc[:,2:].columns.tolist()
        # Check no missing farm number
        assert pd.isna(farm_number_list).any() == False, "Missing Farm number"
        
        # Convert dataframe from wide to long
        df_melted = pd.melt(df_clean, id_vars = ["Category", np.nan], value_vars=farm_number_list, var_name="Farm number")
        #Remove all values with Na as not filled with x
        df_filter = df_melted.dropna(subset = ["value"])
        # Rename columns
        df_filter = df_filter.rename({np.nan:"Activity"}, axis = 1)
        # Remove value column
        df_filter = df_filter.drop("value", axis = 1)

        # Template dataset 
        df_file = pd.DataFrame({  "Farm number":[],"Network name":[], "Start year":[], "Web link":[], "Contact email":[],"Postcode":[], "Category":[], "Activity":[]})
        #Counter used in for loop
        x = 0
        # Loop through each farm in form
        for farm in df_filter['Farm number'].unique():
            df_farm = df_filter[df_filter["Farm number"]== farm]
            # Add relevant info for farm
            df_farm['Postcode'] = postcode_list[x]
            df_farm['Start year'] = year_list[x]
            df_farm['Network name'] = network_name
            df_farm['Web link'] = network_web_link
            df_farm['Contact email'] = network_email
            # Add one to counter to ensure relevant info is added
            x+=1
            df_file = pd.concat([df_file,df_farm])

        df_total = pd.concat([df_total,df_file])

        # Check that columns which should not have NA are complete
        assert df_total[['Activity',"Category", "Farm number", "Postcode"]].isnull().values.any() == False, "Column contains NA values"

# Add on Postcode distric lon lat info
df_total_postcode = pd.merge(df_total, df_postcode_lookup,how = "left", on = "Postcode" )
# Write out to csv
df_total_postcode.to_csv(f"ExportData_{datetime.today().strftime('%Y_%m_%d')}.csv", index = False)

SFN mapping form Agroecology at Scale_KS.xlsx
SFN mapping form Climate Farm Demo IfA_MD_LB__updated.xlsx
SFN mapping form James Hutton Institute_AC.xlsx
SFN mapping form M&S FF_LB.xlsx
SFN mapping form Organic Variety Research_DA.xlsx
